In [ ]:
class Sentence:
    def __init__(self, verb, agent, role, team):
        self.verb = verb
        self.agent = agent
        self.role = role
        self.team = team

In [ ]:
class A:
    def __init__(self, flag):
        self.flag = flag
        self.sentences = []
    def append(self, verb, agent, role, team):
        sentence = Sentence(verb, agent, role, team)
        self.sentences.append(sentence)

In [ ]:
a = A(flag = True)

In [ ]:
a.append((verb, [agent]))

In [ ]:
(1, [(verb, [agent]), (verb, [agent]), ...])